In [1]:
#Dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score


In [2]:
#load in datasets
matches = pd.read_csv("epl_matches.csv", index_col=0) 
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2024,Arsenal
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2024,Arsenal
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2024,Arsenal
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2024,Arsenal
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2024,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2019-04-13,12:30,Premier League,Matchweek 34,Sat,Away,L,0.0,4.0,Tottenham,...,Match Report,NaN,7.0,1.0,18.9,1.0,0,0,2019,Huddersfield Town
36,2019-04-20,15:00,Premier League,Matchweek 35,Sat,Home,L,1.0,2.0,Watford,...,Match Report,NaN,13.0,3.0,18.1,1.0,0,0,2019,Huddersfield Town
37,2019-04-26,20:00,Premier League,Matchweek 36,Fri,Away,L,0.0,5.0,Liverpool,...,Match Report,NaN,5.0,1.0,21.6,0.0,0,0,2019,Huddersfield Town
38,2019-05-05,14:00,Premier League,Matchweek 37,Sun,Home,D,1.0,1.0,Manchester Utd,...,Match Report,NaN,7.0,3.0,19.0,1.0,0,0,2019,Huddersfield Town


In [3]:
#convert date to date-time
matches["date"] = pd.to_datetime(matches["date"])
matches.dtypes

date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                     float64
ga                     float64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                  float64
sh                     float64
sot                    float64
dist                   float64
fk                     float64
pk                       int64
pkatt                    int64
season                   int64
team                    object
dtype: object

In [4]:
# Remove columns that are not needed
#[comp , attendance, captain  , match report, notes, dist]
matches = matches.drop(columns=["comp", "attendance", "captain", "match report", "notes", "dist"])

# remove the Matchweek from the round column and convert to integer
matches["round"] = matches["round"].str.replace("Matchweek", "").astype(int)


In [5]:
# Rearrange columns
# 'season' date, time, round, day, venue, team, 'formation' gf, ga, 'xg', 'xga', 'poss', 'sh', 'sot','fk', 'pk', 'pkatt','result','referee', 'opponent'
matches = matches[['season', 'round', 'date', 'time','day', 'venue', 'team', 'formation', 'gf', 'ga', 'xg', 'xga', 'poss', 'sh', 'sot', 'fk', 'pk', 'pkatt', 'result', 'referee', 'opponent']]
#rename team to Team_1 and opponent to Team_2
matches = matches.rename(columns={"team": "Team_1", "opponent": "Team_2"})


In [6]:
# sort mathces by team1, season, round
matches = matches.sort_values(by=["Team_1", "season", "round"]).reset_index(drop=True)
# create a copy of the matches dataframe
matches_2 = matches.copy()
# sort matches_2 by team_2, season, round
matches_2 = matches_2.sort_values(by=["Team_2", "season", "round"]).reset_index(drop=True)


In [7]:
# add _2 to the columns of matches_2
matches_2.columns = [col + "_2" for col in matches_2.columns]
display(matches)
display(matches_2)

,season,round,date,time,day,venue,Team_1,formation,gf,ga,...,xga,poss,sh,sot,fk,pk,pkatt,result,referee,Team_2
0,2019,1,2018-08-12,16:00,Sun,Home,Arsenal,4-2-3-1,0.0,2.0,...,1.7,42.0,9.0,3.0,0.0,0,0,L,Michael Oliver,Manchester City
1,2019,2,2018-08-18,17:30,Sat,Away,Arsenal,4-2-3-1,2.0,3.0,...,1.6,38.0,15.0,6.0,0.0,0,0,L,Martin Atkinson,Chelsea
2,2019,3,2018-08-25,15:00,Sat,Home,Arsenal,4-2-3-1,3.0,1.0,...,1.1,61.0,17.0,10.0,1.0,0,0,W,Graham Scott,West Ham
3,2019,4,2018-09-02,13:30,Sun,Away,Arsenal,4-2-3-1,3.0,2.0,...,1.0,72.0,17.0,11.0,0.0,0,0,W,Anthony Taylor,Cardiff City
4,2019,5,2018-09-15,15:00,Sat,Away,Arsenal,4-2-3-1,2.0,1.0,...,0.4,63.0,12.0,2.0,1.0,0,0,W,Lee Probert,Newcastle Utd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,2024,24,2024-02-10,15:00,Sat,Home,Wolverhampton Wanderers,3-4-3,0.0,2.0,...,1.9,71.0,17.0,5.0,1.0,0,0,L,Simon Hooper,Brentford
4362,2024,25,2024-02-17,15:00,Sat,Away,Wolverhampton Wanderers,3-4-3,2.0,1.0,...,1.6,29.0,12.0,7.0,0.0,0,0,W,Anthony Taylor,Tottenham
4363,2024,26,2024-02-25,13:30,Sun,Home,Wolverhampton Wanderers,3-4-3,1.0,0.0,...,0.6,63.0,13.0,2.0,0.0,0,0,W,Darren Bond,Sheffield Utd
4364,2024,27,2024-03-02,15:00,Sat,Away,Wolverhampton Wanderers,3-4-3,0.0,3.0,...,2.1,56.0,12.0,3.0,0.0,0,0,L,Tim Robinson,Newcastle Utd


,season_2,round_2,date_2,time_2,day_2,venue_2,Team_1_2,formation_2,gf_2,ga_2,...,xga_2,poss_2,sh_2,sot_2,fk_2,pk_2,pkatt_2,result_2,referee_2,Team_2_2
0,2019,1,2018-08-12,16:00,Sun,Away,Manchester City,4-2-3-1,2.0,0.0,...,0.5,58.0,17.0,8.0,1.0,0,0,W,Michael Oliver,Arsenal
1,2019,2,2018-08-18,17:30,Sat,Home,Chelsea,4-3-3,3.0,2.0,...,2.4,62.0,24.0,11.0,1.0,0,0,W,Martin Atkinson,Arsenal
2,2019,3,2018-08-25,15:00,Sat,Away,West Ham United,4-4-1-1,1.0,3.0,...,1.8,39.0,13.0,5.0,0.0,0,0,L,Graham Scott,Arsenal
3,2019,4,2018-09-02,13:30,Sun,Home,Cardiff City,4-4-2,2.0,3.0,...,1.7,28.0,14.0,3.0,0.0,0,0,L,Anthony Taylor,Arsenal
4,2019,5,2018-09-15,15:00,Sat,Home,Newcastle United,4-4-1-1,1.0,2.0,...,1.0,37.0,4.0,2.0,0.0,0,0,L,Lee Probert,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,2024,24,2024-02-10,15:00,Sat,Away,Brentford,3-5-2,2.0,0.0,...,1.0,29.0,9.0,6.0,0.0,0,0,W,Simon Hooper,Wolves
4362,2024,25,2024-02-17,15:00,Sat,Home,Tottenham Hotspur,4-2-3-1,1.0,2.0,...,1.8,71.0,15.0,4.0,0.0,0,0,L,Anthony Taylor,Wolves
4363,2024,26,2024-02-25,13:30,Sun,Away,Sheffield United,3-5-2,0.0,1.0,...,1.3,37.0,12.0,4.0,0.0,0,0,L,Darren Bond,Wolves
4364,2024,27,2024-03-02,15:00,Sat,Home,Newcastle United,4-3-3,3.0,0.0,...,0.8,44.0,14.0,6.0,1.0,0,0,W,Tim Robinson,Wolves


In [8]:
#concatenate matches and matches_2 dataframes along the columns
matches_combined = pd.concat([matches, matches_2], axis=1)
matches_combined


,season,round,date,time,day,venue,Team_1,formation,gf,ga,...,xga_2,poss_2,sh_2,sot_2,fk_2,pk_2,pkatt_2,result_2,referee_2,Team_2_2
0,2019,1,2018-08-12,16:00,Sun,Home,Arsenal,4-2-3-1,0.0,2.0,...,0.5,58.0,17.0,8.0,1.0,0,0,W,Michael Oliver,Arsenal
1,2019,2,2018-08-18,17:30,Sat,Away,Arsenal,4-2-3-1,2.0,3.0,...,2.4,62.0,24.0,11.0,1.0,0,0,W,Martin Atkinson,Arsenal
2,2019,3,2018-08-25,15:00,Sat,Home,Arsenal,4-2-3-1,3.0,1.0,...,1.8,39.0,13.0,5.0,0.0,0,0,L,Graham Scott,Arsenal
3,2019,4,2018-09-02,13:30,Sun,Away,Arsenal,4-2-3-1,3.0,2.0,...,1.7,28.0,14.0,3.0,0.0,0,0,L,Anthony Taylor,Arsenal
4,2019,5,2018-09-15,15:00,Sat,Away,Arsenal,4-2-3-1,2.0,1.0,...,1.0,37.0,4.0,2.0,0.0,0,0,L,Lee Probert,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,2024,24,2024-02-10,15:00,Sat,Home,Wolverhampton Wanderers,3-4-3,0.0,2.0,...,1.0,29.0,9.0,6.0,0.0,0,0,W,Simon Hooper,Wolves
4362,2024,25,2024-02-17,15:00,Sat,Away,Wolverhampton Wanderers,3-4-3,2.0,1.0,...,1.8,71.0,15.0,4.0,0.0,0,0,L,Anthony Taylor,Wolves
4363,2024,26,2024-02-25,13:30,Sun,Home,Wolverhampton Wanderers,3-4-3,1.0,0.0,...,1.3,37.0,12.0,4.0,0.0,0,0,L,Darren Bond,Wolves
4364,2024,27,2024-03-02,15:00,Sat,Away,Wolverhampton Wanderers,3-4-3,0.0,3.0,...,0.8,44.0,14.0,6.0,1.0,0,0,W,Tim Robinson,Wolves


In [9]:
#drop the Team_2_2 , Team_2 columns
matches_combined = matches_combined.drop(columns=["Team_2_2", "Team_2"])

matches_combined.columns

Index(['season', 'round', 'date', 'time', 'day', 'venue', 'Team_1',
       'formation', 'gf', 'ga', 'xg', 'xga', 'poss', 'sh', 'sot', 'fk', 'pk',
       'pkatt', 'result', 'referee', 'season_2', 'round_2', 'date_2', 'time_2',
       'day_2', 'venue_2', 'Team_1_2', 'formation_2', 'gf_2', 'ga_2', 'xg_2',
       'xga_2', 'poss_2', 'sh_2', 'sot_2', 'fk_2', 'pk_2', 'pkatt_2',
       'result_2', 'referee_2'],
      dtype='object')

In [10]:
#rename the team_1_2 column to team_2
matches_combined = matches_combined.rename(columns={"Team_1_2": "Team_2"})
matches_combined.columns

Index(['season', 'round', 'date', 'time', 'day', 'venue', 'Team_1',
       'formation', 'gf', 'ga', 'xg', 'xga', 'poss', 'sh', 'sot', 'fk', 'pk',
       'pkatt', 'result', 'referee', 'season_2', 'round_2', 'date_2', 'time_2',
       'day_2', 'venue_2', 'Team_2', 'formation_2', 'gf_2', 'ga_2', 'xg_2',
       'xga_2', 'poss_2', 'sh_2', 'sot_2', 'fk_2', 'pk_2', 'pkatt_2',
       'result_2', 'referee_2'],
      dtype='object')

In [11]:
#rename result to result_team_1
matches_combined = matches_combined.rename(columns={"result": "result_team_1"})
#rename result to result_team_1
matches_combined = matches_combined.rename(columns={"result_2": "result_team_2"})

In [12]:
#target will be if team_1 won lost or drawn
conditions = [
    (matches_combined["result_team_1"] == "W"),  # Win condition
    (matches_combined["result_team_1"] == "L"),  # Lose condition
    (matches_combined["result_team_1"] == "D")   # Draw condition
]

values = [1, -1, 0,]  # 1 for win, -1 for lose, 0 for draw
matches_combined["target"] = np.select(conditions, values, default=np.nan)
matches_combined

,season,round,date,time,day,venue,Team_1,formation,gf,ga,...,xga_2,poss_2,sh_2,sot_2,fk_2,pk_2,pkatt_2,result_team_2,referee_2,target
0,2019,1,2018-08-12,16:00,Sun,Home,Arsenal,4-2-3-1,0.0,2.0,...,0.5,58.0,17.0,8.0,1.0,0,0,W,Michael Oliver,-1.0
1,2019,2,2018-08-18,17:30,Sat,Away,Arsenal,4-2-3-1,2.0,3.0,...,2.4,62.0,24.0,11.0,1.0,0,0,W,Martin Atkinson,-1.0
2,2019,3,2018-08-25,15:00,Sat,Home,Arsenal,4-2-3-1,3.0,1.0,...,1.8,39.0,13.0,5.0,0.0,0,0,L,Graham Scott,1.0
3,2019,4,2018-09-02,13:30,Sun,Away,Arsenal,4-2-3-1,3.0,2.0,...,1.7,28.0,14.0,3.0,0.0,0,0,L,Anthony Taylor,1.0
4,2019,5,2018-09-15,15:00,Sat,Away,Arsenal,4-2-3-1,2.0,1.0,...,1.0,37.0,4.0,2.0,0.0,0,0,L,Lee Probert,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,2024,24,2024-02-10,15:00,Sat,Home,Wolverhampton Wanderers,3-4-3,0.0,2.0,...,1.0,29.0,9.0,6.0,0.0,0,0,W,Simon Hooper,-1.0
4362,2024,25,2024-02-17,15:00,Sat,Away,Wolverhampton Wanderers,3-4-3,2.0,1.0,...,1.8,71.0,15.0,4.0,0.0,0,0,L,Anthony Taylor,1.0
4363,2024,26,2024-02-25,13:30,Sun,Home,Wolverhampton Wanderers,3-4-3,1.0,0.0,...,1.3,37.0,12.0,4.0,0.0,0,0,L,Darren Bond,1.0
4364,2024,27,2024-03-02,15:00,Sat,Away,Wolverhampton Wanderers,3-4-3,0.0,3.0,...,0.8,44.0,14.0,6.0,1.0,0,0,W,Tim Robinson,-1.0


In [13]:
#target will be if team_2 won lost or drawn
conditions = [
    (matches_combined["result_team_2"] == "W"),  # Win condition
    (matches_combined["result_team_2"] == "L"),  # Lose condition
    (matches_combined["result_team_2"] == "D")   # Draw condition
]

values = [1, -1, 0,]  # 1 for win, -1 for lose, 0 for draw
matches_combined["target_2"] = np.select(conditions, values, default=np.nan)
matches_combined

,season,round,date,time,day,venue,Team_1,formation,gf,ga,...,poss_2,sh_2,sot_2,fk_2,pk_2,pkatt_2,result_team_2,referee_2,target,target_2
0,2019,1,2018-08-12,16:00,Sun,Home,Arsenal,4-2-3-1,0.0,2.0,...,58.0,17.0,8.0,1.0,0,0,W,Michael Oliver,-1.0,1.0
1,2019,2,2018-08-18,17:30,Sat,Away,Arsenal,4-2-3-1,2.0,3.0,...,62.0,24.0,11.0,1.0,0,0,W,Martin Atkinson,-1.0,1.0
2,2019,3,2018-08-25,15:00,Sat,Home,Arsenal,4-2-3-1,3.0,1.0,...,39.0,13.0,5.0,0.0,0,0,L,Graham Scott,1.0,-1.0
3,2019,4,2018-09-02,13:30,Sun,Away,Arsenal,4-2-3-1,3.0,2.0,...,28.0,14.0,3.0,0.0,0,0,L,Anthony Taylor,1.0,-1.0
4,2019,5,2018-09-15,15:00,Sat,Away,Arsenal,4-2-3-1,2.0,1.0,...,37.0,4.0,2.0,0.0,0,0,L,Lee Probert,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,2024,24,2024-02-10,15:00,Sat,Home,Wolverhampton Wanderers,3-4-3,0.0,2.0,...,29.0,9.0,6.0,0.0,0,0,W,Simon Hooper,-1.0,1.0
4362,2024,25,2024-02-17,15:00,Sat,Away,Wolverhampton Wanderers,3-4-3,2.0,1.0,...,71.0,15.0,4.0,0.0,0,0,L,Anthony Taylor,1.0,-1.0
4363,2024,26,2024-02-25,13:30,Sun,Home,Wolverhampton Wanderers,3-4-3,1.0,0.0,...,37.0,12.0,4.0,0.0,0,0,L,Darren Bond,1.0,-1.0
4364,2024,27,2024-03-02,15:00,Sat,Away,Wolverhampton Wanderers,3-4-3,0.0,3.0,...,44.0,14.0,6.0,1.0,0,0,W,Tim Robinson,-1.0,1.0


In [14]:
def calculate_rolling_stats(matches_combined, window):
    # Sort the DataFrame by team and date
    matches_combined = matches_combined.sort_values(['Team_1', 'season', 'round', 'date'])

    # Calculate rolling statistics for the team
    matches_combined['last_{}_results'.format(window)] = matches_combined.groupby('Team_1')['target'].rolling(window).sum().shift().reset_index(0, drop=True)
    matches_combined['last_{}_gf'.format(window)] = matches_combined.groupby('Team_1')['gf'].rolling(window).sum().shift().reset_index(0, drop=True)
    matches_combined['last_{}_ga'.format(window)] = matches_combined.groupby('Team_1')['ga'].rolling(window).sum().shift().reset_index(0, drop=True)
    matches_combined['last_{}_avg_poss'.format(window)] = matches_combined.groupby('Team_1')['poss'].rolling(window).mean().shift().reset_index(0, drop=True)
    matches_combined['last_{}_avg_sot'.format(window)] = matches_combined.groupby('Team_1')['sot'].rolling(window).mean().shift().reset_index(0, drop=True)
   
    # Sort the DataFrame by team_2 season, round, and date
    matches_combined = matches_combined.sort_values(['Team_2', 'season', 'round', 'date'])

    # Calculate rolling statistics for team_2
    matches_combined['last_{}_results_2'.format(window)] = matches_combined.groupby('Team_2')['target_2'].rolling(window).sum().shift().reset_index(0, drop=True)
    matches_combined['last_{}_gf_2'.format(window)] = matches_combined.groupby('Team_2')['gf'].rolling(window).sum().shift().reset_index(0, drop=True)
    matches_combined['last_{}_ga_2'.format(window)] = matches_combined.groupby('Team_2')['ga'].rolling(window).sum().shift().reset_index(0, drop=True)
    matches_combined['last_{}_avg_poss_2'.format(window)] = matches_combined.groupby('Team_2')['poss_2'].rolling(window).mean().shift().reset_index(0, drop=True)
    matches_combined['last_{}_avg_sot_2'.format(window)] = matches_combined.groupby('Team_2')['sot_2'].rolling(window).mean().shift().reset_index(0, drop=True)

    # Calculate the difference between team and opponent stats
    matches_combined['last_{}_gd'.format(window)] = matches_combined['last_{}_gf'.format(window)] - matches_combined['last_{}_ga'.format(window)]
    matches_combined['opp_last_{}_gd'.format(window)] = matches_combined['last_{}_gf_2'.format(window)] - matches_combined['last_{}_ga_2'.format(window)]
    matches_combined['last_{}_gd_diff'.format(window)] = matches_combined['last_{}_gd'.format(window)] - matches_combined['opp_last_{}_gd'.format(window)]
    matches_combined['last_{}_avg_poss_diff'.format(window)] = matches_combined['last_{}_avg_poss'.format(window)] - matches_combined['last_{}_avg_poss_2'.format(window)]
    matches_combined['last_{}_avg_sot_diff'.format(window)] = matches_combined['last_{}_avg_sot'.format(window)] - matches_combined['last_{}_avg_sot_2'.format(window)]
    matches_combined['last_{}_results_diff'.format(window)] = matches_combined['last_{}_results'.format(window)] - matches_combined['last_{}_results_2'.format(window)]
    return matches_combined


# Calculate rolling statistics for the last 3 games
matches_combined = calculate_rolling_stats(matches_combined, 3)
matches_combined


,season,round,date,time,day,venue,Team_1,formation,gf,ga,...,last_3_results_2,last_3_gf_2,last_3_ga_2,last_3_avg_poss_2,last_3_avg_sot_2,last_3_gd,opp_last_3_gd,last_3_gd_diff,last_3_avg_poss_diff,last_3_avg_sot_diff
2468,2019,1,2018-08-12,16:00,Sun,Away,Manchester City,4-2-3-1,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1084,2019,2,2018-08-18,17:30,Sat,Home,Chelsea,4-3-3,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3931,2019,3,2018-08-25,15:00,Sat,Away,West Ham United,4-4-1-1,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048,2019,4,2018-09-02,13:30,Sun,Home,Cardiff City,4-4-2,2.0,3.0,...,-1.0,6.0,5.0,47.000000,6.333333,-5.0,1.0,-6.0,-3.333333,-4.666667
2908,2019,5,2018-09-15,15:00,Sat,Home,Newcastle United,4-4-1-1,1.0,2.0,...,1.0,6.0,8.0,57.000000,9.000000,-6.0,-2.0,-4.0,-26.666667,-6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,2024,24,2024-02-10,15:00,Sat,Away,Brentford,3-5-2,2.0,0.0,...,0.0,6.0,7.0,39.000000,4.666667,-1.0,-1.0,0.0,-2.666667,-0.666667
3773,2024,25,2024-02-17,15:00,Sat,Home,Tottenham Hotspur,4-2-3-1,1.0,2.0,...,-1.0,8.0,7.0,53.333333,5.333333,0.0,1.0,-1.0,5.333333,0.333333
3366,2024,26,2024-02-25,13:30,Sun,Away,Sheffield United,3-5-2,0.0,1.0,...,1.0,5.0,6.0,45.333333,6.000000,-3.0,-1.0,-2.0,-17.333333,-3.666667
3120,2024,27,2024-03-02,15:00,Sat,Home,Newcastle United,4-3-3,3.0,0.0,...,1.0,3.0,3.0,54.333333,4.666667,3.0,0.0,3.0,1.666667,-1.000000


In [15]:
# calculate the columns with null values
null_columns = matches_combined.columns[matches_combined.isnull().any()]
null_columns

Index(['last_3_results', 'last_3_gf', 'last_3_ga', 'last_3_avg_poss',
       'last_3_avg_sot', 'last_3_results_2', 'last_3_gf_2', 'last_3_ga_2',
       'last_3_avg_poss_2', 'last_3_avg_sot_2', 'last_3_gd', 'opp_last_3_gd',
       'last_3_gd_diff', 'last_3_avg_poss_diff', 'last_3_avg_sot_diff'],
      dtype='object')

In [16]:
# calculate the columns with null values
null_columns = matches_combined.columns[matches_combined.isnull().any()]

# fill the NaN values in the dataframe columns with the median value of those columns

columns_to_fill = null_columns

for column in columns_to_fill:
    median_value = matches_combined[column].median()
    matches_combined[column].fillna(median_value, inplace=True)

matches_combined.head()

,season,round,date,time,day,venue,Team_1,formation,gf,ga,...,last_3_results_2,last_3_gf_2,last_3_ga_2,last_3_avg_poss_2,last_3_avg_sot_2,last_3_gd,opp_last_3_gd,last_3_gd_diff,last_3_avg_poss_diff,last_3_avg_sot_diff
2468,2019,1,2018-08-12,16:00,Sun,Away,Manchester City,4-2-3-1,2.0,0.0,...,0.0,4.0,4.0,49.666667,4.000000,0.0,0.0,0.0,0.000000,0.000000
1084,2019,2,2018-08-18,17:30,Sat,Home,Chelsea,4-3-3,3.0,2.0,...,0.0,4.0,4.0,49.666667,4.000000,0.0,0.0,0.0,0.000000,0.000000
3931,2019,3,2018-08-25,15:00,Sat,Away,West Ham United,4-4-1-1,1.0,3.0,...,0.0,4.0,4.0,49.666667,4.000000,0.0,0.0,0.0,0.000000,0.000000
1048,2019,4,2018-09-02,13:30,Sun,Home,Cardiff City,4-4-2,2.0,3.0,...,-1.0,6.0,5.0,47.000000,6.333333,-5.0,1.0,-6.0,-3.333333,-4.666667
2908,2019,5,2018-09-15,15:00,Sat,Home,Newcastle United,4-4-1-1,1.0,2.0,...,1.0,6.0,8.0,57.000000,9.000000,-6.0,-2.0,-4.0,-26.666667,-6.000000


In [17]:
# setting the venue code

matches_combined["venue_code"] = matches_combined["venue"].astype("category").cat.codes
matches_combined


,season,round,date,time,day,venue,Team_1,formation,gf,ga,...,last_3_gf_2,last_3_ga_2,last_3_avg_poss_2,last_3_avg_sot_2,last_3_gd,opp_last_3_gd,last_3_gd_diff,last_3_avg_poss_diff,last_3_avg_sot_diff,venue_code
2468,2019,1,2018-08-12,16:00,Sun,Away,Manchester City,4-2-3-1,2.0,0.0,...,4.0,4.0,49.666667,4.000000,0.0,0.0,0.0,0.000000,0.000000,0
1084,2019,2,2018-08-18,17:30,Sat,Home,Chelsea,4-3-3,3.0,2.0,...,4.0,4.0,49.666667,4.000000,0.0,0.0,0.0,0.000000,0.000000,1
3931,2019,3,2018-08-25,15:00,Sat,Away,West Ham United,4-4-1-1,1.0,3.0,...,4.0,4.0,49.666667,4.000000,0.0,0.0,0.0,0.000000,0.000000,0
1048,2019,4,2018-09-02,13:30,Sun,Home,Cardiff City,4-4-2,2.0,3.0,...,6.0,5.0,47.000000,6.333333,-5.0,1.0,-6.0,-3.333333,-4.666667,1
2908,2019,5,2018-09-15,15:00,Sat,Home,Newcastle United,4-4-1-1,1.0,2.0,...,6.0,8.0,57.000000,9.000000,-6.0,-2.0,-4.0,-26.666667,-6.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,2024,24,2024-02-10,15:00,Sat,Away,Brentford,3-5-2,2.0,0.0,...,6.0,7.0,39.000000,4.666667,-1.0,-1.0,0.0,-2.666667,-0.666667,0
3773,2024,25,2024-02-17,15:00,Sat,Home,Tottenham Hotspur,4-2-3-1,1.0,2.0,...,8.0,7.0,53.333333,5.333333,0.0,1.0,-1.0,5.333333,0.333333,1
3366,2024,26,2024-02-25,13:30,Sun,Away,Sheffield United,3-5-2,0.0,1.0,...,5.0,6.0,45.333333,6.000000,-3.0,-1.0,-2.0,-17.333333,-3.666667,0
3120,2024,27,2024-03-02,15:00,Sat,Home,Newcastle United,4-3-3,3.0,0.0,...,3.0,3.0,54.333333,4.666667,3.0,0.0,3.0,1.666667,-1.000000,1


In [18]:
# Create numeric codes for each unique 'opponent' value and store them in a new column 'opp_code'.
matches_combined["opp_code"] = matches_combined["Team_2"].astype("category").cat.codes

In [19]:
# Create numeric codes for each unique 'formation value' value and store them in a new column 'formation_code'.
matches_combined["formation_code"] = matches_combined["formation"].astype("category").cat.codes
# Create numeric codes for each unique 'formation value' value and store them in a new column 'formation_code'.
matches_combined["formation_code_2"] = matches_combined["formation_2"].astype("category").cat.codes

In [20]:
# Create numeric codes for each unique 'team' value and store them in a new column 'team_code'.
matches_combined["team_code"] = matches_combined["Team_1"].astype("category").cat.codes

In [21]:
# Extract the hour component from the 'time' column and store it as integers in a new column named 'hour'.
matches_combined["hour"] = matches_combined["time"].str.replace(":.+", "", regex=True).astype("int")


In [22]:
# Add a new column 'day_code' to the 'matches' DataFrame, containing the day of the week from the 'date' column.
matches_combined["day_code"] = matches_combined["date"].dt.dayofweek
matches_combined

,season,round,date,time,day,venue,Team_1,formation,gf,ga,...,last_3_gd_diff,last_3_avg_poss_diff,last_3_avg_sot_diff,venue_code,opp_code,formation_code,formation_code_2,team_code,hour,day_code
2468,2019,1,2018-08-12,16:00,Sun,Away,Manchester City,4-2-3-1,2.0,0.0,...,0.0,0.000000,0.000000,0,0,12,12,16,16,6
1084,2019,2,2018-08-18,17:30,Sat,Home,Chelsea,4-3-3,3.0,2.0,...,0.0,0.000000,0.000000,1,0,16,12,7,17,5
3931,2019,3,2018-08-25,15:00,Sat,Away,West Ham United,4-4-1-1,1.0,3.0,...,0.0,0.000000,0.000000,0,0,17,12,26,15,5
1048,2019,4,2018-09-02,13:30,Sun,Home,Cardiff City,4-4-2,2.0,3.0,...,-6.0,-3.333333,-4.666667,1,0,18,12,6,13,6
2908,2019,5,2018-09-15,15:00,Sat,Home,Newcastle United,4-4-1-1,1.0,2.0,...,-4.0,-26.666667,-6.000000,1,0,17,12,18,15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,2024,24,2024-02-10,15:00,Sat,Away,Brentford,3-5-2,2.0,0.0,...,0.0,-2.666667,-0.666667,0,27,7,4,3,15,5
3773,2024,25,2024-02-17,15:00,Sat,Home,Tottenham Hotspur,4-2-3-1,1.0,2.0,...,-1.0,5.333333,0.333333,1,27,12,4,23,15,5
3366,2024,26,2024-02-25,13:30,Sun,Away,Sheffield United,3-5-2,0.0,1.0,...,-2.0,-17.333333,-3.666667,0,27,7,4,21,13,6
3120,2024,27,2024-03-02,15:00,Sat,Home,Newcastle United,4-3-3,3.0,0.0,...,3.0,1.666667,-1.000000,1,27,16,4,18,15,5


In [23]:
"venue_code", "opp_code", "hour", "day_code", 'team_code'

('venue_code', 'opp_code', 'hour', 'day_code', 'team_code')

In [25]:
# rearrange the columns in the matches_combined dataframe
matches_combined = matches_combined[["venue_code", "opp_code", "hour", "day_code", 'team_code','season', 'round', 'date', 'time', 'day', 'venue',
'Team_1', 'formation', 'formation_code', 'gf', 'ga', 'xg', 'xga', 'poss', 'sh', 'sot', 'fk', 'pk', 'pkatt', 'result_team_1', 'referee', 
'season_2', 'round_2', 'date_2', 'time_2', 'day_2', 'venue_2', 'Team_2', 'formation_2', 'formation_code_2','gf_2', 'ga_2', 'xg_2', 'xga_2', 'poss_2', 'sh_2', 'sot_2', 'fk_2', 'pk_2', 'pkatt_2', 'result_team_2', 'referee_2',
'last_3_results', 'last_3_gf', 'last_3_ga', 'last_3_avg_poss', 'last_3_avg_sot', 'last_3_gd',
'last_3_results_2', 'last_3_gf_2', 'last_3_ga_2', 'last_3_avg_poss_2', 'last_3_avg_sot_2',   'opp_last_3_gd', 
'last_3_gd_diff', 'last_3_avg_poss_diff', 'last_3_avg_sot_diff','target', 'target_2']]

matches_combined
 

,venue_code,opp_code,hour,day_code,team_code,season,round,date,time,day,...,last_3_gf_2,last_3_ga_2,last_3_avg_poss_2,last_3_avg_sot_2,opp_last_3_gd,last_3_gd_diff,last_3_avg_poss_diff,last_3_avg_sot_diff,target,target_2
2468,0,0,16,6,16,2019,1,2018-08-12,16:00,Sun,...,4.0,4.0,49.666667,4.000000,0.0,0.0,0.000000,0.000000,1.0,-1.0
1084,1,0,17,5,7,2019,2,2018-08-18,17:30,Sat,...,4.0,4.0,49.666667,4.000000,0.0,0.0,0.000000,0.000000,1.0,-1.0
3931,0,0,15,5,26,2019,3,2018-08-25,15:00,Sat,...,4.0,4.0,49.666667,4.000000,0.0,0.0,0.000000,0.000000,-1.0,1.0
1048,1,0,13,6,6,2019,4,2018-09-02,13:30,Sun,...,6.0,5.0,47.000000,6.333333,1.0,-6.0,-3.333333,-4.666667,-1.0,1.0
2908,1,0,15,5,18,2019,5,2018-09-15,15:00,Sat,...,6.0,8.0,57.000000,9.000000,-2.0,-4.0,-26.666667,-6.000000,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,0,27,15,5,3,2024,24,2024-02-10,15:00,Sat,...,6.0,7.0,39.000000,4.666667,-1.0,0.0,-2.666667,-0.666667,1.0,-1.0
3773,1,27,15,5,23,2024,25,2024-02-17,15:00,Sat,...,8.0,7.0,53.333333,5.333333,1.0,-1.0,5.333333,0.333333,-1.0,1.0
3366,0,27,13,6,21,2024,26,2024-02-25,13:30,Sun,...,5.0,6.0,45.333333,6.000000,-1.0,-2.0,-17.333333,-3.666667,-1.0,1.0
3120,1,27,15,5,18,2024,27,2024-03-02,15:00,Sat,...,3.0,3.0,54.333333,4.666667,0.0,3.0,1.666667,-1.000000,1.0,-1.0


In [26]:
# drop the columns that are not needed from the matches_combined dataframe
#'referee_2' 'season_2', 'round_2', 'date_2', 'time_2', 'day_2'
matches_combined = matches_combined.drop(columns=['referee_2', 'season_2', 'round_2', 'date_2', 'time_2', 'day_2'])

#sort the dataframe by date most recent first
matches_combined = matches_combined.sort_values(by="date", ascending=False).reset_index(drop=True)
matches_combined


,venue_code,opp_code,hour,day_code,team_code,season,round,date,time,day,...,last_3_gf_2,last_3_ga_2,last_3_avg_poss_2,last_3_avg_sot_2,opp_last_3_gd,last_3_gd_diff,last_3_avg_poss_diff,last_3_avg_sot_diff,target,target_2
0,0,26,14,6,1,2024,29,2024-03-17,14:00,Sun,...,5.0,9.0,46.000000,5.000000,-4.0,2.0,2.000000,0.000000,0.0,0.0
1,1,1,14,6,26,2024,29,2024-03-17,14:00,Sun,...,8.0,7.0,48.000000,5.000000,1.0,1.0,-2.000000,0.000000,0.0,0.0
2,1,23,17,5,10,2024,29,2024-03-16,17:30,Sat,...,3.0,8.0,72.333333,5.000000,-5.0,3.0,-30.666667,0.333333,1.0,-1.0
3,1,3,15,5,5,2024,29,2024-03-16,15:00,Sat,...,8.0,5.0,40.333333,5.333333,3.0,-6.0,15.000000,-2.333333,1.0,-1.0
4,0,15,15,5,20,2024,29,2024-03-16,15:00,Sat,...,8.0,4.0,45.666667,3.000000,4.0,-6.0,-9.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,1,6,15,5,2,2019,1,2018-08-11,15:00,Sat,...,5.0,4.0,60.666667,4.666667,1.0,-1.0,-9.000000,0.000000,1.0,-1.0
4362,0,11,15,5,7,2019,1,2018-08-11,15:00,Sat,...,2.0,7.0,41.666667,6.000000,-5.0,2.0,-9.666667,0.000000,1.0,-1.0
4363,1,7,15,5,11,2019,1,2018-08-11,15:00,Sat,...,4.0,4.0,32.000000,6.000000,0.0,3.0,9.666667,0.000000,-1.0,1.0
4364,1,13,20,4,17,2019,1,2018-08-10,20:00,Fri,...,9.0,4.0,43.000000,2.666667,5.0,-4.0,17.666667,4.000000,1.0,-1.0


In [27]:
#output to csv
matches_combined.to_csv("epl_matches_combined.csv")
